In [34]:
import nltk
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET # ElementTree Library


In [23]:
tree = ET.parse('/Users/cesc/Downloads/DrugBank_database.xml')

In [47]:
root = tree.getroot()
#for i in root:
#    print(i.tag)

for drug in root:    
    if drug.tag == '{http://www.drugbank.ca}drug':
        drug_name = drug.find('name').text
        synonims = drug.find('synonims').text
        print('hola')
        print(drug_name, synonims)


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()

In [18]:
# Set of common words that we don't want to include
common_words = set(stopwords.words('english'))

drugbank_db_mod = []
for entity in drugbank_db:
    tokens = nltk.word_tokenize(entity)
    for token in tokens:
        drugbank_db_mod.append(token)
        
drugbank_db_mod2 = []
for token in drugbank_db_mod:
    if '-' in token:
        split_tokens = token.split('-')
        for token in split_tokens:
            drugbank_db_mod2.append(token)
    else:
        drugbank_db_mod2.append(token)

        
drugbank_db_mod3 = []
for token in drugbank_db_mod2:
    if len(token) > 3 and not token.isnumeric() and token not in drugbank_db_mod3 and token not in common_words:
        drugbank_db_mod3.append(token)

In [19]:
with(open('data/DrugBank_names_DB2.txt', 'w')) as f:
    for entity in drugbank_db_mod3:
        f.write(entity + '\n')    

In [20]:
sentence = 'Hello my name is Ibuprofeno and I like to take Aspirins from now and then Fluimucil and almax'
tokenized_sentence = nltk.word_tokenize(sentence)

for token in tokenized_sentence:
    for drug in drugbank_db_mod3:
        if drug in token:
            print(token, drug)

Ibuprofeno Ibuprofen
like like
